In [ ]:
import httpx

In [ ]:
def snomed_browser(term: str, limit: int = 100, semanticTag: str = "procedure") -> list[dict]:
    """
    Search SNOMED CT for concepts matching the given term.
    """
    base_url = "http://snowstorm:8080"
    edition = "MAIN"

    url = f"{base_url}/{edition}/concepts"

    params = {
        # "term": term,
        "activeFilter": "true",  # recommended filter by SNOMED CT
        "termActive": "true",  # recommended filter by SNOMED CT
        "ecl": f'< 71388002|Procedure| {{{{ term = ("{term}") }}}}',
    }

    print(params)
    with httpx.Client() as client:
        response = client.get(url, params=params)
        response.raise_for_status()
        items = response.json().get("items", [])
        filtered_matches = [
            match
            for match in items
            # if match["definitionStatus"] == "FULLY_DEFINED"
        ]
        filtered_matches = sorted(
            filtered_matches,
            key=lambda x: len(x["fsn"]["term"]),
        )
        return filtered_matches[:limit]

In [ ]:
search_term = "Electroencephalogram"
matches = snomed_browser(search_term)

matches

In [1]:
from medminer.tools.procedure import SNOMEDTool, SNOMEDQueryMethod


tool = SNOMEDTool(base_url="http://snowstorm:8080", edition="MAIN")
tool.forward("Computed Cranial Tomography")

[{'id': '16559621000119108',
  'term': 'CT guided biopsy of neck',
  'fsn': 'Biopsy of neck using computed tomography guidance (procedure)'},
 {'id': '16548951000119100',
  'term': 'Single photon emission computed tomography of brain',
  'fsn': 'Single photon emission computed tomography of brain (procedure)'},
 {'id': '16548901000119104',
  'term': 'Single photon emission computed tomography of kidney',
  'fsn': 'Single photon emission computed tomography of kidney (procedure)'},
 {'id': '16555501000119102',
  'term': 'CT guided injection of celiac plexus',
  'fsn': 'Injection of celiac plexus using computed tomography guidance (procedure)'},
 {'id': '16554411000119104',
  'term': 'PET CT of heart',
  'fsn': 'Positron emission tomography with computed tomography of heart (procedure)'},
 {'id': '16554361000119106',
  'term': 'PET CT of brain',
  'fsn': 'Positron emission tomography with computed tomography of brain (procedure)'},
 {'id': '16549791000119102',
  'term': 'CT guided needle

In [2]:
tool._build_ecl_query("Computed Cranial Tomography", method=SNOMEDQueryMethod.ANY)

'< 71388002|Procedure| {{ term = ("Computed" "Cranial" "Tomography")}}'